In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

# A Simple Classifier

## Problem
1. Finish the derivation of the simple classifier provided in class
2. Use the [Iris Data Set](https://en.wikipedia.org/wiki/Iris_flower_data_set). Create a classifier for the labels "I.setosa" vs "I.versicolor" using 80% of the data. Compute the classification error using the remaining 20%. Then, repeat the problem for the labels "I.virginica" vs "I.versicolor". Report your results in a clear and concise form.

## Getting the Data

[Scikit-Learn](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) has the data already in its library. For comprehensibility, the data was moved to a pandas data frame to match the wikipedia page.

In [2]:
iris_sk=load_iris()
species_lp={0:'I.setosa', 1:'I. versicolor', 2:'I. virginica'}
iris_df={'sepal_length':iris_sk['data'][:,0], 'sepal_width':iris_sk['data'][:,1], 'petal_length':iris_sk['data'][:,2],
        'petal_width':iris_sk['data'][:,3], 'species':iris_sk['target']}
iris_df=pd.DataFrame(data=iris_df)
print(iris_df)

     sepal_length  sepal_width  petal_length  petal_width  species
0             5.1          3.5           1.4          0.2        0
1             4.9          3.0           1.4          0.2        0
2             4.7          3.2           1.3          0.2        0
3             4.6          3.1           1.5          0.2        0
4             5.0          3.6           1.4          0.2        0
..            ...          ...           ...          ...      ...
145           6.7          3.0           5.2          2.3        2
146           6.3          2.5           5.0          1.9        2
147           6.5          3.0           5.2          2.0        2
148           6.2          3.4           5.4          2.3        2
149           5.9          3.0           5.1          1.8        2

[150 rows x 5 columns]


### Training and Validation Data

As mentioned some data needs to be saved for verififcation.

In [3]:
def partition_df(df, ratio):
    N = df.shape[0]
    M = round(N*ratio)
    train = np.zeros((N), dtype=np.bool)
    train[0:M] = 1
    np.random.shuffle(train)
    train_df = df[train]
    val_df = df[~train]
    return train_df, val_df

# Get the two species to classify
test1_df = iris_df.loc[iris_df['species'] < 2]

# Get training and validation sets
train1_df, ver1_df = partition_df(test1_df, 0.8)
print(train1_df)
print(ver1_df)

    sepal_length  sepal_width  petal_length  petal_width  species
0            5.1          3.5           1.4          0.2        0
1            4.9          3.0           1.4          0.2        0
2            4.7          3.2           1.3          0.2        0
3            4.6          3.1           1.5          0.2        0
4            5.0          3.6           1.4          0.2        0
..           ...          ...           ...          ...      ...
95           5.7          3.0           4.2          1.2        1
96           5.7          2.9           4.2          1.3        1
97           6.2          2.9           4.3          1.3        1
98           5.1          2.5           3.0          1.1        1
99           5.7          2.8           4.1          1.3        1

[80 rows x 5 columns]
    sepal_length  sepal_width  petal_length  petal_width  species
10           5.4          3.7           1.5          0.2        0
13           4.3          3.0           1.1          

## Creating the Classifier

In [4]:
# Get the midpoints
cm = train1_df.loc[train1_df['species'] == 0].mean(axis = 0) 
cp = train1_df.loc[train1_df['species'] == 1].mean(axis = 0) 
cm = np.array(cm)[:-1]
cp = np.array(cp)[:-1]
c = (cp + cm)/2

def classify(x, cp, cm, c):
    return np.sign(np.dot(cp - cm, x - c))

# Run classifier on validation data
true = np.array(ver1_df)[:, -1]*2-1 
val = np.array(ver1_df)[:, :-1] 
test = np.array([classify(x, cp, cm, c) for x in val])
     
test_error = 1/ver1_df.shape[0]*np.sum(1/2*abs(test-true))
print('Risk: %s' % str(test_error))

# Run classifier on validation data
true_t = np.array(train1_df)[:, -1]*2-1 
val_t = np.array(train1_df)[:, :-1] 
test_t = np.array([classify(x, cp, cm, c) for x in val_t])
test_error = 1/train1_df.shape[0]*np.sum(1/2*abs(test_t-true_t))
print('Empirical Risk: %s' % str(test_error))

Risk: 0.0
Empirical Risk: 0.0


# Perceptron

Consider a training set $\left\{\left(x_{1}, y_{1}\right), \ldots,\left(x_{n}, y_{n}\right)\right\}$, with $x_{i} \in \mathbb{R}^{d}$ and $y_{i} \in-1,1$.  The perceptron is one of the oldest algorithm in machine learning.  Historical notes are provided [here](https://en.wikipedia.org/wiki/Perceptron). The  perceptron  is  a  linear  classifier $f(x)=w^{T} x$ where $w \in \mathbb{R}^{d}$. The algorithm for computing $w$ is as follows:

Init: $w \leftarrow y_{1} x_{1}$ <br />
&emsp;for $i=2 \ldots n$ do <br />
&emsp;&emsp;if $y_{i} w^{T} x_{i}<0$ then $w \leftarrow w+y_{i} x_{i}$ <br />
&emsp;&emsp;end if <br />
end for

In [23]:
w = val_t[2, :]
for i in range(3, train1_df.shape[0]):
    print(np.dot(w, val_t[i,:]))
    if true_t[i]*np.dot(w, val_t[i,:]) <= 0:
        w += true_t[i]*val_t[i,:]
percept = [np.sign(np.dot(w, x)) for x in val]
risk = np.sum(1/2*np.abs(percept-true))
print(percept)

3031.07
3257.8600000000006
0.0
3014.0099999999998
3158.3599999999997
2744.45
2973.4300000000003
2992.0800000000004
2818.73
3200.75
2974.3899999999994
3283.9399999999996
2970.53
2914.5799999999995
2555.5199999999995
2807.899999999999
2638.1
2686.4199999999996
2699.2199999999993
2619.5599999999995
2422.4599999999996
2402.9399999999996
2632.97
2682.5299999999993
2296.65
2463.48
2018.75
2290.46
2203.6
1947.76
2204.59
2276.6
2164.5799999999995
1878.0999999999997
2104.4299999999994
1919.0099999999995
3030.7499999999995
2852.0299999999993
3036.5099999999993
2848.6999999999994
2621.49
2880.8699999999994
2142.9499999999994
2873.459999999999
2383.41
2157.7499999999995
2651.0599999999995
2773.8299999999995
2445.1599999999994
2890.7199999999993
2631.3499999999995
2544.9099999999994
2687.2299999999996
2441.9099999999994
2800.1399999999994
2629.6399999999994
2819.0699999999993
2750.8099999999995
2854.4799999999996
3005.1799999999994
2718.6099999999997
2399.8899999999994
2387.5399999999995
2520.12999

## 1

Write  the  kernalized  perceptron  algorithm. Provide a pseudo-code.

### Solution

A kernel makes its decision from the equation

\begin{equation}
f_k(x) = \operatorname{sgn}\left(\sum_{i=1}^{m} y_{i} \alpha_{i} k\left(x, x_{i}\right)\right),
\end{equation}

whereas the perceptron's decision function can be expressed as

\begin{equation}
f_p(x) = \operatorname{sgn} \left( w^T x \right)
\end{equation}

$w$ was generated by successively adding terms $y_i x_i$ upon a misclassification condition. Allow the indices of misclassified points to be described by a set $U$ where its elements $u \in \mathbb N$. The decision function, then, can be described by

\begin{equation}
\begin{aligned}
f_p(x) &= \operatorname{sgn} \left( \langle \sum_{i \in U} y_i x_i, x \rangle \right) \\ 
&= \operatorname{sgn} \left( \sum_{i \in U} y_i k \left( x_i, x \right) \right) \\
&= \operatorname{sgn} \left( \sum_{i = 1}^{m} y_i a_i k \left( x_i, x \right) \right) \\
\end{aligned}
\end{equation}

for $a_i$ defined by

\begin{equation}
a_i=\left\{\begin{array}{ll}{1} & {\text { if } i \in U} \\ {0} & {\text { if } i \notin U}\end{array}\right.
\end{equation}

## 2

Write  the  code  for  data  in  2  dimensions,  similarly  than  for  the  simple  classifier. Show 3 examples using 3 different kernels.

# Kernels over $\mathcal{X} = \mathbb R^2$

Let $x = (x_1, x_2) \in \mathbb R^2$ and $y=\left(y_{1}, y_{2}\right) \in \mathbb{R}^{2}$, 

## 1
Let

\begin{equation}
\phi(x)=\left(x_{1}^{2}, \sqrt{2} x_{1} x_{2}, x_{2}^{2}\right)
\end{equation}

Verify that $\phi(x)^{T} \phi(y)=\left(x^{T} y\right)^{2}$.

### Solution
Evaluate the right side, 
\begin{equation}
\begin{aligned}
\left(x^{T} y\right)^{2} &= \left( x_1 y_1+ x_2y_2 \right)^2 \\
&= x_1^2 y_1^2 + 2 x_1 y_1 y_1 y_2 + x^2_2 y^2_2. \\
\end{aligned}
\end{equation}

Evaluate the left side,
\begin{equation}
\begin{aligned}
\phi(x)^{T} \phi(y) &= \left[\begin{array}{llll}{x_1^2} & {\sqrt{2} x_1 x_2} & {x_2^2}\end{array}\right] \left[\begin{array}{c}{y_1^2} \\ {\sqrt{2} y_1 y_2} \\ {y_2^2}\end{array}\right] \\
&= x_1^2 y_1^2 + 2 x_1 y_1 y_1 y_2 + x^2_2 y^2_2. \\
\end{aligned}
\end{equation}

## 2
Find a function $\phi(x): \mathbb{R}^{2} \mapsto \mathbb{R}^{6}$ such that for any $(x, y), \phi(x)^{T} \phi(y)=\left(x^{T} y+1\right)^{2}$.

### Solution

\begin{equation}
\begin{aligned}
\phi(x)^{T} \phi(y) &= \left(x^Ty + 1 \right)^2 \\
&= \left(x^T y \right)^2 + 2x^T y + 1 \\
&= x_1^2 y_1^2 + 2 x_1 y_1 y_1 y_2 + x^2_2 y^2_2 + 2x_1 y_1 + 2 x_2 y_2 + 1
\end{aligned}
\end{equation}

Observing the symmetry,
\begin{equation}
\begin{aligned}
\phi(x) = \left[\begin{array}{c}{x_1^2} \\ {\sqrt{2} x_1 x_2} \\ {x_2^2} \\ {\sqrt{2}x_1} \\ {\sqrt{2} x_2} \\ {1}\end{array}\right]
\end{aligned}
\end{equation}

## 3
Find a function $\phi(x): \mathbb{R}^{2} \mapsto \mathbb{R}^{9}$ such that for any $(x, y), \phi(x)^{T} \phi(y)=\left(x^{T} y+1\right)^{2}$.

### Solution

An obvious choice is

\begin{equation}
\begin{aligned}
\phi(x) = \left[\begin{array}{c}{x_1^2} \\ {\sqrt{2} x_1 x_2} \\ {x_2^2} \\ {\sqrt{2}x_1} \\ {\sqrt{2} x_2} \\ {1} \\ {0} \\ {0} \\ {0}\end{array}\right]
\end{aligned}
\end{equation}

The inner product is unitary invariant. So, any $\phi' = U \phi $ will also satisfy as a solution such that $U^TU=I$ holds.

## 4
Verify that

\begin{equation}
K(x, y)=\left(1+x^{T} y\right)^{d}
\end{equation}

for $d=1,2 \ldots$ is a positive definite kernel.

### Solution

A kernel, $k(x_i, x_j)$ is positive definite if for a set of numbers $a_1, a_2, ..., a_m \in \mathbb R$,

\begin{equation}
\sum_{i=1}^{m} \sum_{j=1}^{m} a_{i} a_{j} K\left(x_{i}, x_{j}\right) \geq 0
\end{equation}

By substitution, the left side is

\begin{equation}
\sum_{i=1}^{m} \sum_{j=1}^{m} a_{i} a_{j} \left(1+x_i^{T} x_j\right)^{d}.
\end{equation}

This permits the binomial expansion,

\begin{equation}
\sum_{i=1}^{m} \sum_{j=1}^{m} a_{i} a_{j} \sum_{k=0}^{d} \binom{d}{k} 1^{d-k} \left(x_i^{T} x_j\right)^k = \sum_{k=0}^{d}  \binom{d}{k} \sum_{i=1}^{m} \sum_{j=1}^{m} a_{i} a_{j} \left(x_i^{T} x_j\right)^k.
\end{equation}

#### Property 

*Given a family of positive definite kernels $\left( K_i \right)_{i \in \mathbb N}$, $K_i: \mathcal X \times \mathcal X \rightarrow \mathbb R$, the sum $\sum_{i=1}^{n} \lambda_{i} K_{i}$ is positive definite given $\lambda_{1}, \ldots, \lambda_{n} \geq 0$.*

Therefore, as $\binom{d}{k} > 0$, to show that $K(x,y)$ is positive definite, it is suffcient to shows that $K'(x,y) = \left(x^{T} y\right)^k$ is positive definite for $k \in \mathbb N$.

\begin{equation}
 \sum_{j=1}^{m} a_{i} a_{j} \left(x_i^{T} x_j\right)^k = \sum_{j=1}^{m} a_{i} a_{j} \prod_{l=1}^{k} \left(x_i^{T} x_j\right).
\end{equation}

#### Property 

*Given a family of positive definite kernels $\left( K_i \right)_{i \in \mathbb N}$, $K_i: \mathcal X \times \mathcal X \rightarrow \mathbb R$, $K\left(\left(x_{1}, \ldots, x_{n}\right),\left(y_{1}, \ldots, y_{n}\right)\right)=\prod_{i=1}^{n} K_{i}\left(x_{i}, y_{i}\right)$ is a positive definite kernel.*

Therefore, $K'(x,y) = \left(x^{T} y\right)^k$ is positive definite, implying that $K(x, y)=\left(1+x^{T} y\right)^{d}$ is also positive definite.

## 5
Can you find a function $\phi: \mathbb{R}^{2} \mapsto H$, where $H$ is an inner product space such that for any $(x, y),<\phi(x), \phi(y)>_{H}=x^{T} y-1$?